In [23]:
import requests
import pandas as pd

def get_exchange_rates(start_date, end_date, base_currency, target_currency):
    url = "https://api.apilayer.com/exchangerates_data/timeseries"
    headers = {
        "apikey": "xG4VP7WvDuXtP5HxdzfyHmPw0rR4Ymza"
    }
    rates = {}
    year = pd.Timedelta(days=365)
    query_date = pd.to_datetime(end_date)
    while query_date >= pd.to_datetime(start_date):
        # Set the start and end dates for the current year
        current_year_start = (query_date - year).strftime('%Y-%m-%d')
        current_year_end = query_date.strftime('%Y-%m-%d')
        params = {'access_key': 'xG4VP7WvDuXtP5HxdzfyHmPw0rR4Ymza',
                  'start_date': current_year_start,
                  'end_date': current_year_end,
                  'base': base_currency,
                  'symbols': target_currency}
        try:
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            rates.update(data['rates'])
        except requests.exceptions.RequestException as e:
            print("Error: ", e)
        query_date -= year
    # Create a DataFrame from the rates dictionary
    df = pd.DataFrame.from_dict(rates, orient='index')
    return df

# Define the parameters for the function
start_date = '2013-12-31'
end_date = '2022-12-31'
base_currency_list = ['CAD', 'CNY', 'EUR', 'GBP', 'INR', 'JPY', 'RUB', 'KRW']
target_currency = 'USD'

# Initialize an empty dataframe to hold the concatenated results
result_df = pd.DataFrame()

# Call the function for each base currency and concatenate the result horizontally
for base_currency in base_currency_list:
    temp_df = get_exchange_rates(start_date, end_date, base_currency, target_currency)
    temp_df.columns = [base_currency]  # Rename the column to the base currency code
    if result_df.empty:
        result_df = temp_df
    else:
        result_df = pd.concat([result_df, temp_df], axis=1)


                 CAD       CNY       EUR       GBP       INR       JPY  \
2013-01-02  1.012672  0.160039  1.320664  1.626879  0.018368  0.011477   
2013-01-03  1.012338  0.160110  1.307521  1.612313  0.018366  0.011447   
2013-01-04  1.013542  0.160139  1.309641  1.611448  0.018275  0.011392   
2013-01-05  1.013007  0.160118  1.309641  1.611448  0.018244  0.011371   
2013-01-06  1.013438  0.160125  1.309898  1.611741  0.018247  0.011361   
...              ...       ...       ...       ...       ...       ...   
2022-12-27  0.739235  0.143676  1.063683  1.202617  0.012081  0.007491   
2022-12-28  0.735318  0.143293  1.062869  1.202971  0.012070  0.007460   
2022-12-29  0.737817  0.143594  1.066155  1.205473  0.012072  0.007522   
2022-12-30  0.737001  0.144963  1.072673  1.210300  0.012085  0.007627   
2022-12-31  0.736947  0.144972  1.072673  1.209800  0.012085  0.007627   

                 RUB       KRW  
2013-01-02  0.033058  0.000940  
2013-01-03  0.033056  0.000940  
2013-01-04  

In [28]:
result_df = result_df.rename(columns={'CAD': 'CAD/USD',
                                      'CNY': 'CNY/USD',
                                      'EUR': 'EUR/USD',
                                      'GBP': 'GBP/USD',
                                      'INR': 'INR/USD',
                                      'JPY': 'JPY/USD',
                                      'RUB': 'RUB/USD',
                                      'KRW': 'KRW/USD'})

result_df.to_excel('daily_exchange_rates.xlsx', index=False)